In [1]:

from utils.trainfiles import TrainFiles
from utils.dataloader import DataLoader
from model.unet import UNet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    print('GPU ready')
else:
    print('Warning: only CPU found')

GPU ready


# Generate trainfiles.json

In [2]:
trainfiles = TrainFiles('trainfiles.yaml',True)
trainfiles.find_files('/home/stephan/Desktop/',['.stk'], 4, 16)

  0%|          | 0/67 [00:00<?, ?it/s]

# Train UNet

In [ ]:
trainfiles = TrainFiles('trainfiles.json',False)
dataloader = DataLoader(trainfiles,batch_size=6,n_pre=2,n_post=2,train_height=400,train_width=400,load_multiple_targets_per_file=True,n_multiple_targets=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet(dataloader.n_pre + dataloader.n_post).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.L1Loss()


In [ ]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    i = 0
    while not dataloader.epoch_done:
        batch_generated = dataloader.get_batch()
        if not batch_generated:
            break
        data = dataloader.X.to(device)
        targets = dataloader.y.to(device)
        model.train()
        outputs = model(data)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'Batch {i+1} (samples {(i+1)*dataloader.batch_size}), Loss: {loss.item()}')
        i += 1
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    dataloader.shuffle_array()

In [ ]:
torch.save(model.state_dict(),'unet.pt')

In [ ]:
from model.modelwrapper import ModelWrapper
wrapper = ModelWrapper('unet.pt',2,2)

In [ ]:
wrapper.denoise_img('/home/stephan/Desktop/students_10.05._old/Bachelorkurs_2023/Coverslip2 evoked spontan R8 ttx.stk')